# 13. Field of Study Top Entities   

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import *
from datetime import date
import pyspark.sql.functions as sf


rootpath = 'wasbs://mag-2018-09-27@magtrainingsource.blob.core.windows.net/mag/'
outputDir = '/output/jiaxin/pyspark/'
conferenceShortName = 'WWW'
conferenceAnalyticsBaseDir = '/output/conferenceAnalytics/'

In [ ]:
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [ ]:
#//Get all papers and transform PaperRank to probability that can be summed. 

@papers =
    SELECT PaperId,
           JournalId,
           ConferenceSeriesId,
           Math.Exp((double) Rank * ( - 1.0) / 1000) AS RankScore,
           EstimatedCitation,
           Year
    FROM Papers;


#//Select target fields of study for ranking by field level

@fosIds =
    SELECT FieldOfStudyId
    FROM FieldsOfStudy
    WHERE FieldsOfStudy.Level IN @fieldOfStudyLevelToInclude;


#//Get all [Paper] -> [Field of Study] relationships along with paper/field details for later calculation

@paperFos =
    SELECT PaperFieldsOfStudy.PaperId,
           PaperFieldsOfStudy.FieldOfStudyId,
           @papers.JournalId,
           @papers.ConferenceSeriesId,
           @papers.RankScore,
           @papers.EstimatedCitation,
           @papers.Year
    FROM PaperFieldsOfStudy
         INNER JOIN
             @fosIds
         ON PaperFieldsOfStudy.FieldOfStudyId == @fosIds.FieldOfStudyId
         INNER JOIN
             @papers
         ON PaperFieldsOfStudy.PaperId == @papers.PaperId;



#//Get all [Journal] -> [Paper] -> [Field of Study] relationships

@paperJournalFos =
    SELECT @paperFos.PaperId,
           @paperFos.FieldOfStudyId,
           (long)@paperFos.JournalId AS EntityId,
           @paperFos.RankScore,
           @paperFos.EstimatedCitation,
           @paperFos.Year
    FROM @paperFos
WHERE @paperFos.JournalId != null;



#//Get all [Conference] -> [Paper] -> [Field of Study] relationships

@paperConferenceFos =
    SELECT @paperFos.PaperId,
           @paperFos.FieldOfStudyId,
           (long)@paperFos.ConferenceSeriesId AS EntityId,
           @paperFos.RankScore,
           @paperFos.EstimatedCitation,
           @paperFos.Year
    FROM @paperFos
WHERE @paperFos.ConferenceSeriesId != null;



#//Get all [Paper] -> [Author] -> [Affiliation] relationships

@paperAuthorAffiliationFos =
    SELECT @paperFos.PaperId,
           @paperFos.FieldOfStudyId,
           PaperAuthorAffiliations.AuthorId,
           PaperAuthorAffiliations.AffiliationId,
           @paperFos.RankScore,
           @paperFos.EstimatedCitation,
           @paperFos.Year
    FROM @paperFos
         JOIN
             PaperAuthorAffiliations
         ON @paperFos.PaperId == PaperAuthorAffiliations.PaperId;



#//Get all [Paper] -> [Author] relationships

@paperAuthorFos =
    SELECT @paperAuthorAffiliationFos.PaperId,
           @paperAuthorAffiliationFos.FieldOfStudyId,
           @paperAuthorAffiliationFos.AuthorId AS EntityId,
           @paperAuthorAffiliationFos.RankScore,
           @paperAuthorAffiliationFos.EstimatedCitation,
           @paperAuthorAffiliationFos.Year
    FROM @paperAuthorAffiliationFos;



#//Get all [Paper] -> [Affiliation] relationships

@paperAffiliationFos =
    SELECT @paperAuthorAffiliationFos.PaperId,
           @paperAuthorAffiliationFos.FieldOfStudyId,
           (long)@paperAuthorAffiliationFos.AffiliationId AS EntityId,
           @paperAuthorAffiliationFos.RankScore,
           @paperAuthorAffiliationFos.EstimatedCitation,
           @paperAuthorAffiliationFos.Year
    FROM @paperAuthorAffiliationFos
    WHERE @paperAuthorAffiliationFos.AffiliationId != null;



#//Union all the relationships we got above to form [Paper] -> [Entity] -> [Field of Study]

@paperEntityFos =
    SELECT @paperJournalFos. *,
           "Journal" AS EntityType
    FROM @paperJournalFos
        UNION ALL
    SELECT @paperConferenceFos. *,
           "Conference" AS EntityType
    FROM @paperConferenceFos 
        UNION ALL 
    SELECT @paperAuthorFos.*, 
         "Author" AS EntityType 
    FROM @paperAuthorFos 
        UNION ALL 
    SELECT @paperAffiliationFos.*, 
           "Affiliation" AS EntityType 
         FROM @paperAffiliationFos;



#//Calcuate All time Rank by Estimated Citations for HIndex calcuation in later step

@paperEntityFosAll =
    SELECT @paperEntityFos. *,
           ROW_NUMBER() OVER(PARTITION BY @paperEntityFos.FieldOfStudyId, @paperEntityFos.EntityId ORDER BY @paperEntityFos.EstimatedCitation DESC) AS CitationRank,
           "All" AS TimeRange
    FROM @paperEntityFos;



#//Calcuate 10yr Rank by Estimated Citations for HIndex calcuation in later step

@paperEntityFos10yr =
    SELECT @paperEntityFos. *,
           ROW_NUMBER() OVER(PARTITION BY @paperEntityFos.FieldOfStudyId, @paperEntityFos.EntityId ORDER BY @paperEntityFos.EstimatedCitation DESC) AS CitationRank,
           "10yr" AS TimeRange
    FROM @paperEntityFos
    WHERE @paperEntityFos.Year >= @thisYear - 10;



#//Calcuate 5yr Rank by Estimated Citations for HIndex calcuation in later step

@paperEntityFos5yr =

    SELECT @paperEntityFos. *,

           ROW_NUMBER() OVER(PARTITION BY @paperEntityFos.FieldOfStudyId, @paperEntityFos.EntityId ORDER BY @paperEntityFos.EstimatedCitation DESC) AS CitationRank,

           "5yr" AS TimeRange

    FROM @paperEntityFos

    WHERE @paperEntityFos.Year >= @thisYear - 5;





//Union all time range result into a single table

@paperEntityFosTimeRange =

    SELECT *

    FROM @paperEntityFosAll

        UNION ALL

    SELECT *

    FROM @paperEntityFos10yr 

        UNION ALL 

    SELECT * FROM @paperEntityFos5yr;





//Calculate Total RankScore, EstimatedCitation and HIndex for each [Field Of Study] -> [Entity] -> [TimeRange]

@entityFosStats =

    SELECT @paperEntityFosTimeRange.FieldOfStudyId,

           @paperEntityFosTimeRange.EntityId,

           @paperEntityFosTimeRange.TimeRange,

           ANY_VALUE(@paperEntityFosTimeRange.EntityType) AS EntityType,

           (double)SUM(@paperEntityFosTimeRange.RankScore) AS RankScore,

           SUM(@paperEntityFosTimeRange.EstimatedCitation) AS EstimatedCitation,

           MAX(@paperEntityFosTimeRange.EstimatedCitation >= @paperEntityFosTimeRange.CitationRank ? @paperEntityFosTimeRange.CitationRank : 0) AS HIndex 

    FROM @paperEntityFosTimeRange

    GROUP BY @paperEntityFosTimeRange.FieldOfStudyId,

             @paperEntityFosTimeRange.EntityId,

             @paperEntityFosTimeRange.TimeRange;





//Rank each [Field Of Study] -> [Entity] -> [TimeRange] by Total RankScore, EstimatedCitation and HIndex

@entityFosStats =

    SELECT @entityFosStats. *,

           RANK() OVER(PARTITION BY @entityFosStats.FieldOfStudyId, @entityFosStats.EntityType, @entityFosStats.TimeRange ORDER BY RankScore DESC) AS RankScore_Rank,

           RANK() OVER(PARTITION BY @entityFosStats.FieldOfStudyId, @entityFosStats.EntityType, @entityFosStats.TimeRange ORDER BY EstimatedCitation DESC) AS EstimatedCitation_Rank,

           RANK() OVER(PARTITION BY @entityFosStats.FieldOfStudyId, @entityFosStats.EntityType, @entityFosStats.TimeRange ORDER BY HIndex DESC) AS HIndex_Rank

    FROM @entityFosStats;







//Keep only top k entities for each field of study. 

@entityFosStats =

    SELECT FieldOfStudyId,

           EntityId,

           EntityType,

           TimeRange,

           Math.Log(RankScore)*1000*-1.0 AS Rank, //Restore rank to its orginal format           

           EstimatedCitation,

           HIndex

    FROM @entityFosStats

    WHERE @entityFosStats.RankScore_Rank <= @topKEntities 

          OR @entityFosStats.EstimatedCitation_Rank <= @topKEntities 

          OR @entityFosStats.HIndex_Rank <= @topKEntities;





OUTPUT @entityFosStats

TO @entityStats_fos_outpath

USING Outputters.Tsv(outputHeader : true);





//output all [FieldOfStudyId, Name] for easier lookup in visuals

@fieldOfStudyDetails =

    SELECT FieldOfStudyId,

           NormalizedName AS Name

    FROM FieldsOfStudy;





OUTPUT @fieldOfStudyDetails

TO @fosDetails_outpath

USING Outputters.Tsv(outputHeader : true);





//Get all [EntityId, Name] for easier lookup in visuals

@entityDetails =

    SELECT Journals.JournalId AS EntityId,

           Journals.DisplayName AS Name

    FROM Journals

        UNION ALL

    SELECT ConferenceSeries.ConferenceSeriesId AS EntityId,

           ConferenceSeries.NormalizedName AS Name

    FROM ConferenceSeries

        UNION ALL 

    SELECT Affiliations.AffiliationId AS EntityId, 

           Affiliations.DisplayName AS Name

    FROM Affiliations 

        UNION ALL 

    SELECT Authors.AuthorId AS EntityId, 

           Authors.DisplayName AS Name 

    FROM Authors;





//Get all distinct EntityIds

@distinctEntityIds =

    SELECT DISTINCT EntityId

    FROM @entityFosStats;





//Only output [EntityId, Name] for top K entities

@entityDetails =

    SELECT @entityDetails.EntityId,

           @entityDetails.Name

    FROM @entityDetails

         INNER JOIN

             @distinctEntityIds

         ON @entityDetails.EntityId == @distinctEntityIds.EntityId;





OUTPUT @entityDetails

TO @entityDetails_outpath

USING Outputters.Tsv(outputHeader : true);

In [ ]:
# output all [FieldOfStudyId, Name] for easier lookup in visuals
fieldsOfStudy = FieldsOfStudy.selectExpr("FieldOfStudyId", "NormalizedName as Name")

fieldsOfStudy.write.csv(outputDir + "fieldsOfStudy.csv", mode='overwrite', header='true')

In [ ]:
sc.stop()